In [7]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from lang_chain.llm import llm


In [9]:
response = llm.invoke("你好")
print(response.content)

您好！很高兴见到您！我是您的AI助手，有什么我可以帮您的吗？😊


In [12]:
stream = llm.stream("你好")
for chunk in stream:
    print(chunk.content, end="")

你好！很高兴见到你 😊  
有什么可以帮你的吗？无论是问题、建议，还是想聊聊天，我都在这里等你！

In [ ]:
def print_stream(stream):
    for chunk in stream:
        print(chunk.content, end="")


In [18]:
from langchain.tools import tool


@tool
def calculate(expression: str) -> str:
    """执行数学计算"""
    return str(eval(expression))


llm_with_tools = llm.bind_tools([calculate])

stream = llm_with_tools.stream("你有什么工具可以用啊？")
for chunk in stream:
    print(chunk.content, end="")


我目前有一个工具可以使用：

**数学计算工具 (calculate)**
- 功能：执行数学计算
- 参数：expression（必需）- 需要计算的数学表达式，类型为字符串

例如，你可以让我计算：
- 简单的加减乘除：`2 + 3 * 4`
- 复杂的表达式：`(1 + 2) * (3 + 4) / 2`
- 平方、开方等：`sqrt(16)` 或 `2^3`

有什么数学计算需要我帮忙的吗？

In [22]:
response = llm_with_tools.invoke("你调用函数计算一下 2+2")

print(response.content)


我来帮您计算 2+2。



In [25]:
prompts = ["问题1"]
answers = llm.batch(prompts)

In [26]:
for answer in answers:
    print(answer.content)

您好！我看到您提到了“问题1”，但似乎没有提供具体的问题内容。为了更好地帮助您，请您详细说明一下您想问什么问题。例如：

- 您是想讨论某个特定主题（如技术、生活、学习等）吗？
- 这是一个系列问题中的第一个吗？如果有后续问题，也请一并提供。
- 或者，您是想让我协助解决某个具体任务或查询？

请补充更多细节，我会尽快为您提供详细的解答或建议！ 😊


In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 第1步：创建提示词模板
# 这也是一个 Runnable，有 invoke、stream 等方法
prompt = ChatPromptTemplate.from_template("将这句话翻译成{target_language}：{text}")
parser = StrOutputParser()

translation_chain = prompt | llm | parser
result = translation_chain.invoke(
    {"target_language": "中文", "text": "Hello, how are you?"}
)
print(result)


这句话的中文翻译是：  
**“你好，最近怎么样？”**  

### 翻译说明：
1. **Hello** → **“你好”**  
   - 这是中文中最通用的问候语，适用于正式或非正式场合。
2. **how are you?** → **“最近怎么样？”**  
   - 字面直译是“你好吗？”，但中文日常交流中更常用“最近怎么样？”来表达对近况的关心，更自然流畅。

### 其他常见译法：
- **更简洁版**：  
  “你好吗？”（直译，略显生硬，但语法正确）  
- **更口语化**：  
  “嗨，最近咋样？”（适合朋友、熟人之间）  
- **更正式版**：  
  “您好，一切可好？”（适用于商务或长辈）

根据使用场景选择最合适的表达即可 😊


In [ ]:
from langchain_core.runnables import RunnableLambda


# 创建自定义处理函数
def extract_translation(text: str) -> str:
    """从模型输出中去除额外的引号和空格"""
    return text.strip().strip('"').strip("'")


# 创建 Lambda Runnable
cleaner = RunnableLambda(extract_translation)

# 在管道中插入清理步骤
translation_chain = prompt | llm | parser | cleaner

result = translation_chain.invoke({"target_language": "法语", "text": "Good morning!"})

print(result)
# 如果模型输出 "Bonjour!" 带引号，cleaner 会去除它们

Good morning!" 的法语翻译是：

**Bonjour!**

### 补充说明：
1. **使用场景**：  
   "Bonjour" 是法语中最常用的早晨问候语，适用于任何场合（正式/非正式），时间范围大约从日出到中午（约12点前）。

2. **文化提示**：  
   在法国，"Bonjour" 是日常社交的必备用语。进入商店、餐厅或与人打招呼时，必须先说 "Bonjour"，否则可能被视为无礼。

3. **其他表达**：  
   - 非常正式场合（如商务会议）：**Bonjour Madame / Monsieur**（早上好，女士/先生）  
   - 亲密朋友间：**Salut!**（你好，但更通用，不分早晚）

4. **发音**：  
   /bɔ̃.ʒuʁ/（类似"崩日呵"，"r" 小舌音）

### 例句：
- **Good morning! How are you?**  
  → Bonjour! Comment allez-vous?  
- **Good morning, Mr. Dupont.**  
  → Bonjour, Monsieur Dupont.  

☀️ 记住：在法国，一句 "Bonjour" 是开启任何交流的钥匙！


In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

# 1. 支持的语言
SUPPORTED_LANGUAGES = {
    "en": "English",
    "es": "Spanish",
    "fr": "French",
    "de": "German",
    "zh": "Chinese",
    "ja": "Japanese",
}


# 2. 验证语言的函数
def validate_language(language_code: str) -> str:
    """验证语言代码是否支持"""
    if language_code.lower() not in SUPPORTED_LANGUAGES:
        raise ValueError(
            f"不支持的语言: {language_code}. "
            f"支持的语言: {', '.join(SUPPORTED_LANGUAGES.keys())}"
        )
    return SUPPORTED_LANGUAGES[language_code.lower()]


# 3. 翻译提示词
translation_prompt = ChatPromptTemplate.from_template("""
你是一个专业的翻译家。
请将以下文本翻译成{target_language}。
只输出翻译后的文本，不需要其他说明。

原文：{text}
翻译：
""")


# 4. 质量检查函数
def check_translation_quality(translation: str) -> dict:
    """检查翻译是否为空"""
    if not translation.strip():
        raise ValueError("翻译结果为空，请重试")
    return {"translation": translation, "quality_check_passed": True}


parser = StrOutputParser()

# 6. 构建 Pipeline
translation_chain = (
    # 验证语言
    RunnableLambda(
        lambda x: {**x, "target_language": validate_language(x["language_code"])}
    )
    # 提示词格式化
    | translation_prompt
    # 调用模型
    | llm
    # 解析输出
    | parser
    # 质量检查
    | RunnableLambda(check_translation_quality)
)

# 7. 使用 Pipeline
try:
    result = translation_chain.invoke(
        {"language_code": "dd", "text": "The quick brown fox jumps over the lazy dog"}
    )
    print(f"翻译结果: {result['translation']}")
except ValueError as e:
    print(f"错误: {e}")

# 输出：
# 翻译结果: El rápido zorro marrón salta sobre el perro perezoso

错误: 不支持的语言: dd. 支持的语言: en, es, fr, de, zh, ja


In [40]:
# 1. 流式输出（逐字输出）
print("=== 流式翻译 ===")
for chunk in translation_chain.stream(
    {"language_code": "fr", "text": "LangChain is amazing!"}
):
    if isinstance(chunk, dict):
        print(chunk["translation"], end="", flush=True)
    else:
        print(chunk, end="", flush=True)

print("\n")

# 3. 异步调用（非阻塞）


async def async_translate():
    result = await translation_chain.ainvoke(
        {"language_code": "zh", "text": "Artificial Intelligence"}
    )
    return result


# 在异步上下文中运行
# result = asyncio.run(async_translate())

=== 流式翻译 ===
LangChain est incroyable !



In [42]:
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

# 1. 翻译阶段
translation_prompt = ChatPromptTemplate.from_template("""
Translate the following text to {target_language}:
Text: {text}
Translation:
""")

translator = translation_prompt | llm | StrOutputParser()

# 2. 评估阶段
evaluation_prompt = ChatPromptTemplate.from_template("""
Please evaluate this translation:
Original: {original_text}
Translation: {translated_text}
Target Language: {target_language}

Provide your evaluation in JSON format with:
- accuracy (1-10)
- naturalness (1-10)
- completeness (1-10)
- overall_quality (1-10)
- feedback (string)

JSON:
""")

evaluator = evaluation_prompt | llm | JsonOutputParser()


# 3. 完整流程
def full_translation_pipeline(original_text: str, target_language: str) -> dict:
    """翻译 + 评估"""

    # 翻译
    translated_text = translator.invoke(
        {"target_language": target_language, "text": original_text}
    )

    # 评估
    evaluation = evaluator.invoke(
        {
            "original_text": original_text,
            "translated_text": translated_text,
            "target_language": target_language,
        }
    )

    # 返回完整结果
    return {
        "original": original_text,
        "translated": translated_text,
        "language": target_language,
        "evaluation": evaluation,
    }


# 4. 使用
result = full_translation_pipeline(
    "Machine learning is a subset of artificial intelligence", "Spanish"
)

print(f"原文: {result['original']}")
print(f"翻译: {result['translated']}")
print(f"评分: {result['evaluation']['overall_quality']}/10")
print(f"反馈: {result['evaluation']['feedback']}")

原文: Machine learning is a subset of artificial intelligence
翻译: **Translation:**  
"Aprendizaje automático es un subconjunto de inteligencia artificial."  

### Notes:
1. **"Machine learning"** se traduce como **"aprendizaje automático"** (término estándar en español, reconocido por la RAE).  
2. **"Subset"** se traduce como **"subconjunto"** (término matemático/tecnológico común).  
3. **"Artificial intelligence"** se traduce como **"inteligencia artificial"**.  
4. La estructura gramatical en español mantiene el orden lógico del original, usando "es" para "is".  
5. No se usan mayúsculas en los términos técnicos (excepto al inicio de la oración), ya que no son nombres propios.
评分: 10/10
反馈: La traducción es impecable. Utiliza términos técnicos estándar en español reconocidos por la RAE ('aprendizaje automático', 'subconjunto', 'inteligencia artificial'), mantiene la estructura gramatical lógica del original, y respeta las convenciones de capitalización del español. La traducción es p